In [ ]:
print("hello")

In [ ]:
!pip install tensorflow
!pip install keras
!pip install gym
!pip install keras-rl2

gym.openai.com

In [ ]:
import gymnasium as gym
import random
import numpy as np

setup environment

In [ ]:
env = gym.make("CartPole-v1")
states = env.observation_space.shape[0]
actions = env.action_space.n

In [ ]:
episodes = 10
for episode in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        env.render()
        action = random.choice(range(actions))
        next_state, reward, done, truncate, info = env.step(action)
        total_reward += reward

    print(f"Episode: {episode + 1}, Total Reward: {total_reward}")

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam


In [ ]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [ ]:
model = build_model(states,actions)

#building agent with keras-rl

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def agent_build(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=10000, window_length = 1)
    dqn = DQNAgent(model= model, memory=memory, policy= policy,
                   nb_actions= actions, nb_steps_warmup=10, target_model_update=1e-2)
    
    return dqn

In [ ]:
dqn = agent_build(model,actions)
dqn.compile(Adam(1e-3),metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False,verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=True)
print(np.mean(scores.history['episode_reward']))

#save and reload model

In [ ]:
dqn.save_weights('dqn_weights.h5f',overwrite=True)

In [ ]:
env = gym.make('CartPole-v1')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states,actions)
dqn = agent_build(model,actions)
dqn.compile(Adam(1e-3), metrics=['mae'])

dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=10, visualize=True)